# Jardín Ornamental (Métodos Sincronización)

AXEL DANIEL MALVÁEZ FLORES

GABRIEL ZADQUIEL PERALTA RIONDA

In [22]:
import multiprocess as mp
import time

In [23]:
#Creamos las funciones que ejecutaran nuestros procesos.
def entrada1(val, n):
  for i in range(n):
    val.value += 1 #Value es una clase y tenemos que acceder a su atributo value.
  
# Con locks
def entrada1c(val, n,lock):
  for i in range(n):
    lock.acquire()
    valor = val.value
    val.value = valor + 1 #Value es una clase y tenemos que acceder a su atributo value.
    lock.release()

# Con banderas
def entrada1b(val, bandera, n):
  while bandera.value == 1:
    if bandera.value == 0:
      break

  bandera.value = 1
  for i in range(n):
    val.value += 1 #Value es una clase y tenemos que acceder a su atributo value.
  bandera.value = 0

# Con turnos solo para dos procesos
def entrada1t(val,turno, n):
  while turno.value != 1:
    if turno.value == 1:
      break
  
  for i in range(n):
    val.value += 1 #Value es una clase y tenemos que acceder a su atributo value.
  turno.value = 2

## Implementación Secuencial

In [24]:
def cuenta_personas(n):
    cuenta = 0
    for i in range(n):
        cuenta = cuenta + 1    
    return cuenta

## Implementación Naive

In [25]:
#Creamos los procesos
val = mp.Value('i',0) #Creamos un objeto de tipo value

p1 = mp.Process(target=entrada1,args=(val,20,))
p2 = mp.Process(target=entrada1,args=(val,20,))

#Inicializamos los procesos.
p1.start()
p2.start()

#Esperamos a que terminen los procesos.
p1.join()
p2.join()

print(val.value)

40


## Crash

Esto se torna incorrecto a partir de incrementar los ciclos de nuestro ```for```.

In [26]:
#Creamos los procesos
val = mp.Value('i',0) #Creamos un objeto de tipo value

p1 = mp.Process(target=entrada1,args=(val,10000//2,))
p2 = mp.Process(target=entrada1,args=(val,10000//2,))

#Inicializamos los procesos.
p1.start()
p2.start()

#Esperamos a que terminen los procesos.
p1.join()
p2.join()

print(val.value)

6349


## Con procesos

In [27]:
def jardin_procesos(n_pros=2, n_iter=20):
    val = mp.Value('i',0) #Creamos un objeto de tipo value

    #Creamos los procesos
    procesos = []
    for i in range(n_pros):
        procesos.append(mp.Process(target=entrada1,args=(val,n_iter,)))

    #Inicializamos los procesos.
    for i in range(n_pros):
        procesos[i].start()

    #Esperamos a que terminen los procesos.
    for i in range(n_pros):
        procesos[i].join()

    return val.value

In [28]:
print(jardin_procesos())
print(jardin_procesos(4, 40//4))
a = jardin_procesos(10, 100000//10)
print(f'Respuesta obtenida: {a} y respuesta esperada: {100000}')

40
40
Respuesta obtenida: 19044 y respuesta esperada: 100000


In [29]:
def calc_tiempo(funcion, n):
    start = time.time()
    valor = funcion(n)
    end = time.time()
    tiempo = end - start
    return tiempo

def calc_tiempo_p(funcion, procesos, iter):
    start = time.time()
    valor = funcion(procesos, iter)
    end = time.time()
    tiempo = end - start
    return tiempo

def calc_tiempo_t(funcion, iter):
    start = time.time()
    valor = funcion(iter)
    end = time.time()
    tiempo = end - start
    return tiempo

In [30]:
pro = 10
it = 10000
print('Tiempo para nuestro algoritmo secuencial', calc_tiempo(cuenta_personas, it))
print(f'Tiempo para nuestro algoritmo con {pro} procesos', calc_tiempo_p(jardin_procesos, pro, it//pro))

Tiempo para nuestro algoritmo secuencial 0.000865936279296875
Tiempo para nuestro algoritmo con 10 procesos 0.1991901397705078


## Implementación Jardín con candados

La solución que presenta utilizar candados implica que se suspende la multitarea durante la sección 
crítica, es decir estamos perdiendo la velocidad de la concurrencia.

In [31]:
def jardin_procesos_candados(n_procesos=2, n_iter=20):
    val = mp.Value('i',0) #Creamos un objeto de tipo value
    lock = mp.Lock()
    #Creamos los procesos
    procesos = []
    for i in range(n_procesos):
        procesos.append(mp.Process(target=entrada1c,args=(val,n_iter,lock)))

    #Inicializamos los procesos.
    for i in range(n_procesos):
        procesos[i].start()

    #Esperamos a que terminen los procesos.
    for i in range(n_procesos):
        procesos[i].join()
    
    return val.value

fallas y tiempos

In [32]:
jardin_procesos_candados(5, 1000000//5)

1000000

In [33]:
pro = 5
it = 100000
print('Tiempo que tarda con Locks:', calc_tiempo_p(jardin_procesos_candados,pro, it// pro))
print('Tiempo que tarda sin Locks:', calc_tiempo_p(jardin_procesos,pro, it// pro))


Tiempo que tarda con Locks: 1.5032892227172852
Tiempo que tarda sin Locks: 1.7642860412597656


## Implementación con banderas

In [34]:
def jardin_bandera(n_procesos=2, n_iter=20):
  # Creamos la bandera y el value
  bandera = mp.Value('i',0)
  val = mp.Value('i',0)

  procesos = []
  for i in range(n_procesos):
    procesos.append(mp.Process(target=entrada1b,args=(val, bandera, n_iter)))

  for i in range(n_procesos):
    procesos[i].start()

  for i in range(n_procesos):
    procesos[i].join()
  
  return val.value

tiempo que tarda

In [35]:
pro = 3
it = 1000000
print(f'Tiempo para nuestro algoritmo con {pro} procesos y usando banderas', calc_tiempo_p(jardin_bandera, pro, it//pro))

Tiempo para nuestro algoritmo con 3 procesos y usando banderas 4.643049001693726


## Implementación con banderas

In [36]:
def jardin_turnos(n_iter):
  turno = mp.Value('i',1)
  val = mp.Value('i',0) #Creamos un objeto de tipo value
  p1 = mp.Process(target=entrada1t,args=(val,turno, n_iter))
  p2 = mp.Process(target=entrada1t,args=(val,turno, n_iter))  
  p1.start()
  p2.start()
  p1.join()
  p2.join()
  return val.value

tiempo que tarda

In [40]:
n = 100000
print(f'Tiempo para nuestro algoritmo con {pro} procesos y usando turnos', calc_tiempo_t(jardin_turnos, n//2))

Tiempo para nuestro algoritmo con 3 procesos y usando turnos 0.7005190849304199


## Implementación Peterson